In [35]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pylab as plt
from gensim.models import LdaModel, CoherenceModel
from gensim import corpora
from scipy import stats as st

In [2]:
mutations = pd.read_table('../data/all_muts_pass_and_polyphen.txt', sep=' ')
gene_exp = pd.read_table('../data/all_tcga_gene_exp.txt', sep=' ')
maf = pd.read_table('../data/all_tcga_maf.txt', sep=' ')
cin_loc = pd.read_table('../data/cin_locations.txt')
sanchez_vega = pd.read_table('../data/sanchez_vega_pws_1026.csv', sep=';')

/scratch/63866778/ipykernel_6812/1516295756.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  mutations = pd.read_table('../data/all_muts_pass_and_polyphen.txt', sep=' ')


In [3]:
file = pd.read_table('../steps/discretized_9.features', dtype=str)

In [4]:
lda = LdaModel.load(f'../steps/gensim/lda/lda_t5_f10_sym.model')
df = file.drop(['Sample', 'Chr'], axis = 1)
listedDf = df.values.tolist()
dirichlet_dict = corpora.Dictionary(listedDf)
bow_corpus = [dirichlet_dict.doc2bow(text) for text in listedDf]

In [5]:
def create_heatmap(model, df):
    topics = lda.get_document_topics(bow = bow_corpus, minimum_probability = 0)
    
    new_df = pd.DataFrame(df['Sample'])
    new_df['Chr'] = df['Chr']
    new_df['Chr'] = pd.to_numeric(new_df['Chr'])
    
    columns = [[],[],[],[],[]]
    for topic in topics:
        for w in topic:
            columns[w[0]].append(w[1])
    
    new_df['1'] = columns[0]
    new_df['2'] = columns[1]
    new_df['3'] = columns[2]
    new_df['4'] = columns[3]
    new_df['5'] = columns[4]
    new_df = new_df.sort_values(by='Chr')
    return new_df


In [6]:
weights_df = create_heatmap(lda, file)

In [7]:
mut = weights_df.reset_index(drop=True)
mut = mut.rename(columns = {'Sample': 'sampleID'})
mut= mut.drop_duplicates(subset = ["sampleID"])
mut = pd.merge(left = mut, right = maf[['sampleID', 'mutatated_genes']], on='sampleID')

gene_list = ['TP53', 'PTEN', 'NF1', 'MYC', 'CDK12', 'CCNE1', 'BRCA2', 'BRCA1']
# gene_list = list(cin_loc['Gene'])

for gene in gene_list:
    mut[gene] = np.where(mut['mutatated_genes'].str.contains(gene), True, False)


mut

,sampleID,Chr,1,2,3,4,5,mutatated_genes,TP53,PTEN,NF1,MYC,CDK12,CCNE1,BRCA2,BRCA1
0,TCGA-C4-A0F7,1,0.167465,0.020485,0.771171,0.020551,0.020326,"HNF1A, TP53",True,False,True,False,False,False,False,False
1,TCGA-29-1711,1,0.446566,0.020526,0.020810,0.491331,0.020766,TP53,True,False,False,False,False,False,False,False
2,TCGA-C4-A0F6,1,0.020450,0.020774,0.020426,0.917627,0.020724,"COL2A1, COL2A1, NFE2L2, CASP8",False,False,False,False,False,False,False,False
3,TCGA-EW-A2FW,1,0.020342,0.020318,0.918107,0.020621,0.020612,LRP1B,False,False,False,False,False,False,False,False
4,TCGA-29-1764,1,0.020553,0.020558,0.918049,0.020403,0.020438,TP53,True,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4535,TCGA-D1-A1O8,1,0.020814,0.020417,0.917677,0.020620,0.020472,"PTEN, PIK3CA, CTNNB1",False,True,False,False,False,False,False,False
4536,TCGA-XR-A8TG,1,0.918228,0.020298,0.020518,0.020532,0.020423,"TP53, TP53",True,False,False,False,False,False,False,False
4537,TCGA-FR-A8YC,1,0.020383,0.918060,0.020599,0.020329,0.020628,"PDE4DIP, CBL, CBL, COL2A1, PTPRB, GRIN2A, CREB...",False,False,False,False,False,False,False,False
4538,TCGA-CV-A6JM,1,0.020362,0.020585,0.020571,0.020822,0.917661,"KMT2D, TP53, CSMD3, EXT1, NOTCH1, CDKN2A",True,False,False,False,False,False,False,False


In [8]:
gene_expT = gene_exp.transpose()
gene_expT['Sample'] = gene_expT.index
gene_expT

,0,1,2,3,4,5,6,7,8,9,...,18358,18359,18360,18361,18362,18363,18364,18365,18366,Sample
TCGA-19-1787,5.5438,2.8974,5.2327,1.23330,2.90900,2.7826,2.64180,5.58980,3.62890,3.25410,...,-9.9658,2.77000,5.46500,-9.9658,3.0446,-9.9658,4.98420,-9.9658,-6.50640,TCGA-19-1787
TCGA-S9-A7J2,5.3709,-1.7809,4.6972,2.33660,1.51150,1.5514,1.14470,3.40340,5.46730,4.46340,...,-9.9658,2.66700,5.40130,-9.9658,3.1908,-9.9658,2.80760,-9.9658,-4.29340,TCGA-S9-A7J2
TCGA-G3-A3CH,5.4422,-3.6370,4.3111,1.93840,-0.19515,1.6659,8.60775,4.64695,4.74575,2.05375,...,-9.9658,-0.32265,3.93430,-9.9658,0.9153,-9.9658,1.01890,-9.9658,-9.96580,TCGA-G3-A3CH
TCGA-EK-A2RE,4.7077,-9.9658,5.8585,2.08100,2.86220,-0.2159,3.10790,4.89340,4.03130,3.07910,...,-9.9658,-0.23280,4.73400,-9.9658,6.1416,-9.9658,-1.21420,-9.9658,-9.96580,TCGA-EK-A2RE
TCGA-44-6778,3.9800,-6.7119,5.3509,2.68705,1.63215,5.5077,4.90205,5.10775,4.21175,3.97765,...,-9.9658,1.26620,4.28085,-7.0004,3.4035,-9.9658,1.88605,-9.9658,-1.40405,TCGA-44-6778
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-95-7947,5.8370,-1.5522,6.3884,3.82080,3.46870,3.3264,5.79500,6.34180,5.77450,4.81610,...,-9.9658,1.22710,5.69460,-9.9658,6.0492,-9.9658,1.29920,-9.9658,-9.96580,TCGA-95-7947
TCGA-VQ-AA6F,3.2988,-9.9658,5.1526,2.59480,2.22060,1.3846,4.04620,5.09640,6.39900,6.31540,...,-9.9658,0.82460,4.27060,-5.5735,5.6473,-9.9658,-1.99420,-9.9658,-9.96580,TCGA-VQ-AA6F
TCGA-BR-8588,3.2959,-3.1714,5.1244,2.40900,2.97470,2.6278,4.69600,5.04840,3.35490,3.33500,...,-9.9658,0.08540,4.28330,-9.9658,2.9128,-9.9658,1.51150,-9.9658,-9.96580,TCGA-BR-8588
TCGA-24-2254,6.5220,-1.6850,5.6372,2.52880,2.98390,3.1507,6.21080,5.74070,4.91320,4.99150,...,-9.9658,0.43400,5.31290,-9.9658,3.3883,-9.9658,3.42100,-9.9658,-9.96580,TCGA-24-2254


In [26]:
exp = weights_df.reset_index(drop=True).drop(['Chr'], axis = 1).drop_duplicates(subset = ["Sample"])
exp.columns = ['Sample', 'Sig1', 'Sig2', 'Sig3', 'Sig4', 'Sig5']
exp = pd.merge(left = exp, right = gene_expT, on = 'Sample')

In [27]:
exp.index = exp['Sample']
exp = exp.drop(['Sample'], axis = 1)
exp

,Sig1,Sig2,Sig3,Sig4,Sig5,0,1,2,3,4,...,18357,18358,18359,18360,18361,18362,18363,18364,18365,18366
Sample,,,,,,,,,,,,,,,,,,,,,
TCGA-C4-A0F7,0.167465,0.020485,0.771171,0.020551,0.020326,5.2068,-5.5735,6.7803,2.2663,3.8360,...,-9.9658,-9.9658,0.2029,5.8958,-9.9658,4.4122,-9.9658,-4.0350,-9.9658,-9.9658
TCGA-DJ-A3US,0.020812,0.020565,0.308761,0.020382,0.629481,4.9199,-9.9658,5.3320,2.9948,1.4441,...,-9.9658,-9.9658,0.2998,5.4202,-9.9658,4.2396,-9.9658,1.6093,-9.9658,-5.0116
TCGA-TS-A8AI,0.020648,0.020348,0.917657,0.020578,0.020768,3.6335,-5.5735,5.6562,1.9302,2.6302,...,-9.9658,-9.9658,-0.0725,4.6404,-9.9658,5.4002,-9.9658,2.4035,-9.9658,-9.9658
TCGA-29-1711,0.446566,0.020526,0.020810,0.491331,0.020766,4.8640,-9.9658,6.3248,2.7051,2.6533,...,-9.9658,-9.9658,0.0854,4.2548,-9.9658,5.5069,-9.9658,2.7292,-9.9658,-9.9658
TCGA-TS-A8AF,0.547853,0.020593,0.020681,0.020679,0.390194,3.7898,-9.9658,5.4909,1.9712,1.6466,...,-9.9658,-9.9658,1.1184,4.1482,-4.6082,5.0418,-9.9658,2.8681,-9.9658,-9.9658
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-FR-A8YC,0.020383,0.918060,0.020599,0.020329,0.020628,3.7867,-3.0469,4.3328,2.5707,1.8078,...,-9.9658,-9.9658,0.7664,4.2578,-9.9658,1.1833,-9.9658,1.5064,-9.9658,1.5709
TCGA-ZF-A9RC,0.470747,0.020440,0.467646,0.020697,0.020470,5.5289,-9.9658,5.8215,1.9895,3.1924,...,-9.9658,-9.9658,1.7532,5.1575,-2.8262,5.3463,-6.5064,-0.7588,-9.9658,-9.9658
TCGA-AB-2896,0.020415,0.020616,0.020681,0.020828,0.917460,-0.8339,-9.9658,5.1879,3.0688,4.0207,...,-9.9658,-9.9658,2.3898,4.0251,-9.9658,0.8246,-9.9658,0.5370,-9.9658,-9.9658


In [28]:
exp_final = exp.drop(['Sig1', 'Sig2', 'Sig3', 'Sig4', 'Sig5'], axis = 1)
exp_final

,0,1,2,3,4,5,6,7,8,9,...,18357,18358,18359,18360,18361,18362,18363,18364,18365,18366
Sample,,,,,,,,,,,,,,,,,,,,,
TCGA-C4-A0F7,5.2068,-5.5735,6.7803,2.2663,3.8360,0.0854,4.7318,4.6136,3.9682,3.0774,...,-9.9658,-9.9658,0.2029,5.8958,-9.9658,4.4122,-9.9658,-4.0350,-9.9658,-9.9658
TCGA-DJ-A3US,4.9199,-9.9658,5.3320,2.9948,1.4441,3.5461,5.1851,5.2533,4.0926,4.0046,...,-9.9658,-9.9658,0.2998,5.4202,-9.9658,4.2396,-9.9658,1.6093,-9.9658,-5.0116
TCGA-TS-A8AI,3.6335,-5.5735,5.6562,1.9302,2.6302,2.2299,7.2257,6.5179,2.9857,3.2915,...,-9.9658,-9.9658,-0.0725,4.6404,-9.9658,5.4002,-9.9658,2.4035,-9.9658,-9.9658
TCGA-29-1711,4.8640,-9.9658,6.3248,2.7051,2.6533,2.2421,3.8749,5.1165,3.7571,4.3428,...,-9.9658,-9.9658,0.0854,4.2548,-9.9658,5.5069,-9.9658,2.7292,-9.9658,-9.9658
TCGA-TS-A8AF,3.7898,-9.9658,5.4909,1.9712,1.6466,3.2127,7.7617,6.0199,3.3105,3.6691,...,-9.9658,-9.9658,1.1184,4.1482,-4.6082,5.0418,-9.9658,2.8681,-9.9658,-9.9658
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-FR-A8YC,3.7867,-3.0469,4.3328,2.5707,1.8078,1.7912,2.9600,4.7377,3.4007,3.2587,...,-9.9658,-9.9658,0.7664,4.2578,-9.9658,1.1833,-9.9658,1.5064,-9.9658,1.5709
TCGA-ZF-A9RC,5.5289,-9.9658,5.8215,1.9895,3.1924,0.7058,2.7161,5.4460,2.3732,2.5585,...,-9.9658,-9.9658,1.7532,5.1575,-2.8262,5.3463,-6.5064,-0.7588,-9.9658,-9.9658
TCGA-AB-2896,-0.8339,-9.9658,5.1879,3.0688,4.0207,5.6282,3.7877,4.9440,4.7555,5.3733,...,-9.9658,-9.9658,2.3898,4.0251,-9.9658,0.8246,-9.9658,0.5370,-9.9658,-9.9658


In [41]:
correlation_df = gene_exp[['TCGA-19-1787']]

In [42]:
correlation_df

,TCGA-19-1787
0,5.5438
1,2.8974
2,5.2327
3,1.2333
4,2.9090
...,...
18362,3.0446
18363,-9.9658
18364,4.9842
18365,-9.9658


In [43]:
exp_final

,0,1,2,3,4,5,6,7,8,9,...,18357,18358,18359,18360,18361,18362,18363,18364,18365,18366
Sample,,,,,,,,,,,,,,,,,,,,,
TCGA-C4-A0F7,5.2068,-5.5735,6.7803,2.2663,3.8360,0.0854,4.7318,4.6136,3.9682,3.0774,...,-9.9658,-9.9658,0.2029,5.8958,-9.9658,4.4122,-9.9658,-4.0350,-9.9658,-9.9658
TCGA-DJ-A3US,4.9199,-9.9658,5.3320,2.9948,1.4441,3.5461,5.1851,5.2533,4.0926,4.0046,...,-9.9658,-9.9658,0.2998,5.4202,-9.9658,4.2396,-9.9658,1.6093,-9.9658,-5.0116
TCGA-TS-A8AI,3.6335,-5.5735,5.6562,1.9302,2.6302,2.2299,7.2257,6.5179,2.9857,3.2915,...,-9.9658,-9.9658,-0.0725,4.6404,-9.9658,5.4002,-9.9658,2.4035,-9.9658,-9.9658
TCGA-29-1711,4.8640,-9.9658,6.3248,2.7051,2.6533,2.2421,3.8749,5.1165,3.7571,4.3428,...,-9.9658,-9.9658,0.0854,4.2548,-9.9658,5.5069,-9.9658,2.7292,-9.9658,-9.9658
TCGA-TS-A8AF,3.7898,-9.9658,5.4909,1.9712,1.6466,3.2127,7.7617,6.0199,3.3105,3.6691,...,-9.9658,-9.9658,1.1184,4.1482,-4.6082,5.0418,-9.9658,2.8681,-9.9658,-9.9658
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-FR-A8YC,3.7867,-3.0469,4.3328,2.5707,1.8078,1.7912,2.9600,4.7377,3.4007,3.2587,...,-9.9658,-9.9658,0.7664,4.2578,-9.9658,1.1833,-9.9658,1.5064,-9.9658,1.5709
TCGA-ZF-A9RC,5.5289,-9.9658,5.8215,1.9895,3.1924,0.7058,2.7161,5.4460,2.3732,2.5585,...,-9.9658,-9.9658,1.7532,5.1575,-2.8262,5.3463,-6.5064,-0.7588,-9.9658,-9.9658
TCGA-AB-2896,-0.8339,-9.9658,5.1879,3.0688,4.0207,5.6282,3.7877,4.9440,4.7555,5.3733,...,-9.9658,-9.9658,2.3898,4.0251,-9.9658,0.8246,-9.9658,0.5370,-9.9658,-9.9658


In [ ]:
correlation_df

for signature in range(1, 6):
    for gene in range(0, 18367):
        correlation_df['Signature 1'][gene] = st.pearsonr(exp[f'Sig{signature}'], exp_final[])